In [4]:
from abc import ABC, abstractmethod
from collections import deque
from typing import List, Dict, Any, Deque
from dataclasses import dataclass
from datetime import datetime


@dataclass
class MarketDataPoint:
	"""
	Represents a single market data tick.
	"""
	timestamp: datetime
	symbol: str
	price: float



In [5]:

# =========================
# Strategy Interface
# =========================
class Strategy(ABC):
    """
    Abstract base class for trading strategies.
    Each strategy consumes one MarketDataPoint (tick)
    and returns a list of trading signals.
    """

    @abstractmethod
    def generate_signals(self, tick: MarketDataPoint) -> List[Dict[str, Any]]:
        """
        Generate trading signals based on a single tick.

        Must be implemented by subclasses.
        """
        pass


# =========================
# Helper: Trading Decision
# =========================
def moving_average_decision(price: float, avg_price: float) -> str:
    """
    Simple moving-average decision rule.

    If price > average  -> BUY
    If price < average  -> SELL
    Else                -> HOLD

    Time Complexity: O(1)
    Space Complexity: O(1)
    """
    if price > avg_price:
        return "BUY"
    elif price < avg_price:
        return "SELL"
    return "HOLD"



In [6]:
# =========================================================
# Strategy 1: Naive Moving Average (Full History)
# =========================================================
class NaiveMovingAverageStrategy(Strategy):
    """
    NAIVE MOVING AVERAGE STRATEGY

    Idea:
      Keep ALL historical prices.
      For every new tick, recompute the average from scratch:
          avg = sum(all_prices) / len(all_prices)

    --- Theoretical Complexity ---
    Let i = number of ticks processed so far.

    Per tick:
      - append price: amortized O(1)
      - sum(history): O(i) because it loops over i elements
      - division + comparisons: O(1)
      => total per tick: O(i) ~ worst-case O(n)

    Over n ticks:
      O(1 + 2 + 3 + ... + n) = O(n^2)

    Space:
      - stores full price history list of length n => O(n)

    Memory note:
      This grows linearly with dataset size and will be much larger than windowed.
    """

    def __init__(self) -> None:
        # Space: O(n) over runtime because this list grows with every tick
        self.price_history: List[float] = []

    def generate_signals(self, tick: MarketDataPoint) -> List[Dict[str, Any]]:
        # Append is amortized O(1) time; list may occasionally resize.
        self.price_history.append(tick.price)

        # sum(self.price_history) is O(i):
        # Python must iterate over the entire list to compute the sum.
        total = sum(self.price_history)

        # len(...) is O(1) for lists (stored length)
        avg_price = total / len(self.price_history)  # division is O(1)

        # Decision is O(1)
        signal = moving_average_decision(tick.price, avg_price)

        # Constructing a small dict is O(1) (bounded keys/values)
        return [{
            "timestamp": tick.timestamp,
            "symbol": tick.symbol,
            "price": tick.price,
            "moving_average": avg_price,
            "signal": signal,
            "strategy": "NaiveMovingAverageStrategy"
        }]


In [12]:


# =========================================================
# Strategy 2: Windowed Moving Average (Fixed Window k)
# =========================================================
class WindowedMovingAverageStrategy(Strategy):
    """
    WINDOWED MOVING AVERAGE STRATEGY

    Idea:
      Maintain only the last k prices (a sliding window),
      and keep a running sum to compute average in O(1).

      window = deque(last k prices)
      running_sum = sum(window)

    --- Theoretical Complexity ---
    Let k be the fixed window size (e.g., 10).

    Per tick:
      - append to deque: O(1)
      - possibly popleft from deque: O(1)
      - update running_sum: O(1)
      - compute average: O(1)
      - decision: O(1)
      => total per tick: O(1)

    Over n ticks:
      O(n)

    Space:
      - deque holds at most k prices: O(k)
      - running_sum is a float: O(1)
      => total: O(k)

    Memory note:
      Unlike naive, memory does NOT grow with n. It stays bounded by k.
    """

    def __init__(self, window_size: int = 10) -> None:
        if window_size <= 0:
            raise ValueError("window_size must be positive")

        self.window_size = window_size

        # deque stores at most k floats => O(k) space
        self.window: Deque[float] = deque()

        # O(1) space
        self.running_sum: float = 0.0

    def generate_signals(self, tick: MarketDataPoint) -> List[Dict[str, Any]]:
        # Add new price:
        # deque append is O(1)
        self.window.append(tick.price)

        # running sum update is O(1)
        self.running_sum += tick.price

        # Maintain fixed window size:
        # len(deque) is O(1)
        if len(self.window) > self.window_size:
            # popleft is O(1) for deque
            oldest = self.window.popleft()
            # update running sum O(1)
            self.running_sum -= oldest

        # Average computed in O(1) from running_sum
        avg_price = self.running_sum / len(self.window)

        # Decision O(1)
        signal = moving_average_decision(tick.price, avg_price)

        return [{
            "timestamp": tick.timestamp,
            "symbol": tick.symbol,
            "price": tick.price,
            "moving_average": avg_price,
            "signal": signal,
            "strategy": f"WindowedMovingAverageStrategy(k={self.window_size})"
        }]


# =========================================================
# Comparison helper: Complexity Summary
# =========================================================
COMPLEXITY_SUMMARY = {
    "NaiveMovingAverageStrategy": {
        "per_tick_time": "O(i) (worst O(n)) due to sum(full_history)",
        "total_time": "O(n^2) across n ticks",
        "space": "O(n) stores full history",
        "memory_behavior": "grows linearly with number of ticks",
    },
    "WindowedMovingAverageStrategy": {
        "per_tick_time": "O(1) constant-time updates",
        "total_time": "O(n) across n ticks",
        "space": "O(k) stores fixed window",
        "memory_behavior": "bounded by window size k (does not grow with n)",
    },
}

print(COMPLEXITY_SUMMARY)

{'NaiveMovingAverageStrategy': {'per_tick_time': 'O(i) (worst O(n)) due to sum(full_history)', 'total_time': 'O(n^2) across n ticks', 'space': 'O(n) stores full history', 'memory_behavior': 'grows linearly with number of ticks'}, 'WindowedMovingAverageStrategy': {'per_tick_time': 'O(1) constant-time updates', 'total_time': 'O(n) across n ticks', 'space': 'O(k) stores fixed window', 'memory_behavior': 'bounded by window size k (does not grow with n)'}}
